# Implementierung eines Reinforcement-Learning-Projekts

## Das Problem
Wir sind Betreiber eine Spedition und sind zuständig für die Belieferung von Supermärkten. Dafür muss Ware zwischen Lagern und den Märkten innerhalb unserer Stadt transportiert werden. Unser innovativer LKW ist selbstfahrend und bekommt lediglich den Auftrag an einer Station (Lager oder Supermarkt) Ware einzusammeln und an einer anderen Station wieder abzuliefern.

## Definitionen
Wir bedienen zwei Lager und zwei Supermärkte.

Lager:
0. A-Lager (A)
1. B-Lager (B)

Supermarkt:
2. C-Markt (C)
3. D-Markt (D)

Diese verteilen sich folgendermaßen in unserer Stadt:

<img src="img/stadt.png" alt="stadt" width="400"/>

Die Stadt ist eine 6x6-Quadratestadt und mit 36 Positionen, die die Koordinaten $(0,0)$ bis $(5,5)$ haben. Der LKW kann sich frei in der Stadt bewegen, aber nicht durch die Grünstreifen fahren.

Die Anzahl der Zustände ergibt sich folgendermaßen:
* 6 x 6 Positionen
* 4 Orte zu denen die Ware gebracht werden kann (A bis D bzw. 0 bis 3)
* 5 Orte, an denen sich die Ware befindet (A bis D bzw. 0 bis 3 und im LKW (Position Nr.4))

\\[ 6 \cdot 6 \cdot 4 \cdot 5 = 720 \texttt{ mögliche Zustände}\\]

Die Aktionen, die der LKW ausführen kann sind:
0. nach Norden fahren
1. nach Osten fahren
2. nach Süden fahren
3. nach Westen fahren
4. Ware einsammeln
5. Ware abladen

Dabei kann er folgende Belohnungen (und Abzüge) erhalten:
* Ware korrekt abliefern: +20
* Ware falsch einsammeln/abliefern: -10
* Pro Schritt: -1

## Definitionen implementieren

In [ ]:
import copy
import random

In [ ]:
city = [
    "+-----------+",
    "|A: : |B: : |",
    "|_: : | : :_|",
    "| : : : : : |",
    "| : | : :_:_|",
    "| | : : : :D|",
    "|C| : | : : |",
    "+-----------+",
]
num_rows = 6
num_cols = 6
rows = [row for row in range(0, num_rows)]
cols = [col for col in range(0, num_cols)]
num_actions = 6
actions = {action for action in range(0, num_actions)}
stations = [(0,0), (0,3), (5,0), (4,5)]
walls = {
    ((0,2), (0,3)), #vertical
    ((1,2), (1,3)),
    ((3,1), (3,2)),
    ((4,0), (4,1)),
    ((5,0), (5,1)),
    ((5,2), (5,3)),
    ((1,0), (2,0)), # horizonal
    ((1,5), (2,5)),
    ((3,4), (4,4)),
    ((3,5), (4,5))
}

In [ ]:
possible_actions = dict()

for row in rows:
    for col in cols:
        possible_actions[(row, col)] = copy.deepcopy(actions)
for key in possible_actions:
    (row, col) = key
    if row == 0:
        possible_actions[key].remove(0)
    if row == 5:
        possible_actions[key].remove(2)
    if col == 0:
        possible_actions[key].remove(3)
    if col == 5:
        possible_actions[key].remove(1)
    if (row, col) not in stations:
        possible_actions[key].remove(4)
        possible_actions[key].remove(5)
    if ((row, col), (row, col + 1)) in walls:
        possible_actions[key].remove(1)
    if ((row, col - 1), (row, col)) in walls:
        possible_actions[key].remove(3)
    if ((row, col), (row + 1, col)) in walls:
        possible_actions[key].remove(2)
    if ((row - 1, col), (row, col)) in walls:
        possible_actions[key].remove(0)

In [ ]:
# state((row_lkw, col_lkw), position ware (0-4), ziel ware(0-3) )
def transition_function(state, action):
    position_lkw, position_goods, position_goal = state
    if action not in possible_actions[position_lkw]:
        return state
    
    if action == 0:
        row, col = position_lkw
        position_lkw = (row - 1, col)
    elif action == 1:
        row, col = position_lkw
        position_lkw = (row, col + 1) 
    elif action == 2:
        row, col = position_lkw
        position_lkw = (row + 1, col)
    elif action == 3:
        row, col = position_lkw
        position_lkw = (row, col - 1)
    elif action == 4:
        position_goods = 4
    elif action == 5:
        if position_goods != 4:
            return state
        position_goods = stations.index(position_lkw)
            
    return (position_lkw, position_goods, position_goal)     

In [ ]:
def reward_function(state, action):
    global counter_wrong_actions
    # Es gibt Abzug auch wenn die Aktion gar nicht möglich ist (beispiel gegen Wand fahren)
    reward = -1 # Pro Schritt
    position_lkw, position_goods, position_goal = state
    if action not in possible_actions[position_lkw]:
        counter_wrong_actions += 1
        return reward
    
    if action == 4:
        station_lkw = stations.index(position_lkw)
        if position_goods != station_lkw: # Ware falsch einsammeln
            reward -= 10
    elif action == 5:
        station_lkw = stations.index(position_lkw)
        if position_goal != station_lkw: # Ware falsch abliefern
            reward -= 10
        if position_goal == station_lkw: # Ware korrekt abliefern
            reward += 20
            
    return reward

In [ ]:
def transport_goods(start_state):
    current_value = 0
    state = start_state
    position_lkw, position_goods, position_goal = state
    while position_goods != position_goal:
        action = random.randrange(0, num_actions, 1)
        print("Action: "+ str(action))
        current_value += reward_function(state, action)
        state = transition_function(state, action)
        print("State: " + str(state))
        print("Current value: " + str(current_value))
        position_lkw, position_goods, position_goal = state

In [ ]:
start_state = ((3,2), 2, 1)
global counter_wrong_actions
counter_wrong_actions = 0
transport_goods(start_state)
print(counter_wrong_actions)